In [1]:
#
# raichu.py : Play the game of Raichu
#
# PLEASE PUT YOUR NAMES AND USER IDS HERE!
#
# Based on skeleton code by D. Crandall, Oct 2021
#

In [2]:
import sys
import time
import numpy as np

w = white pichu // W = white pikachu // @ = white raichu  
b = black pichu //B = Black pikachu // $ = black raichu  

pichus move
* one square diagonally forward
* jump over another pichu and land 2 squares diagonally forward

pikcahus move
* 1 or two squares forward, left or right
* jump over 1 pichu/pikachu and land either two or 3 squares foward left or right

pichus and pikachus turn into raichus if they reach the end of the board (row N for white, row 1 for black)
raichu moves
* any number of squares forward, backward, left, right, diagonally
* jump over 1 pichu/pikachu/raichu and land any number of squares away


In [3]:
initial_board = '........W.W.W.W..w.w.w.w................b.b.b.b..B.B.B.B........'
N = 8

In [4]:
def printable_board(board, N):
    return "\n".join(board[i:i+N] for i in range(0, len(board), N))

In [5]:
print(printable_board(initial_board, 8))


........
W.W.W.W.
.w.w.w.w
........
........
b.b.b.b.
.B.B.B.B
........


In [6]:
def board_to_grid(board, N):
    return np.array(list(board)).reshape(N, N)

In [7]:
def move_on_board(pos, N):
    return 0 <= pos[0] < N and 0<= pos[1] < N

In [8]:
board = board_to_grid(initial_board, N)
board

array([['.', '.', '.', '.', '.', '.', '.', '.'],
       ['W', '.', 'W', '.', 'W', '.', 'W', '.'],
       ['.', 'w', '.', 'w', '.', 'w', '.', 'w'],
       ['.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.'],
       ['b', '.', 'b', '.', 'b', '.', 'b', '.'],
       ['.', 'B', '.', 'B', '.', 'B', '.', 'B'],
       ['.', '.', '.', '.', '.', '.', '.', '.']], dtype='<U1')

In [9]:
test_board = board.copy()
test_board[(4,1)]='w'
test_board[(1,1)]='b'
test_board[(6,2)]='w'
board = test_board
board

array([['.', '.', '.', '.', '.', '.', '.', '.'],
       ['W', 'b', 'W', '.', 'W', '.', 'W', '.'],
       ['.', 'w', '.', 'w', '.', 'w', '.', 'w'],
       ['.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', 'w', '.', '.', '.', '.', '.', '.'],
       ['b', '.', 'b', '.', 'b', '.', 'b', '.'],
       ['.', 'B', 'w', 'B', '.', 'B', '.', 'B'],
       ['.', '.', '.', '.', '.', '.', '.', '.']], dtype='<U1')

In [10]:
#select player
player = 'b'

#set player variables
if player == 'w':
    forward, backward, last_row  = 1, -1, N-1
    pichu, pikachu, raichu = 'w', 'W', '@'
    opp_pichu, opp_pikachu, opp_raichu = 'b', 'B', '$'

if player == 'b':
    forward, backward, last_row  = -1, 1, 0
    pichu, pikachu, raichu = 'b', 'B', '$'
    opp_pichu, opp_pikachu, opp_raichu = 'w', 'W', '@'

In [11]:
def possible_pichu_moves(board):
    pichu_boards = []

    #find player pichus
    row, col = np.where(board == pichu )
    all_pichus = np.column_stack((row,col))

    for pichu_pos in all_pichus:
        # check one diagonal ahead in each direction
        diag_1_l = np.array([forward*1,1])
        diag_1_r = np.array([forward*1,-1])
        diag_1 = np.vstack([np.add(pichu_pos, diag_1_l),
                            np.add(pichu_pos, diag_1_r)])
        diag_1_moves = [tuple(move) for move in diag_1]

        # if space one diagional ahead is empty, add to possible moves list
        to_empty_space = []
        for move in diag_1_moves:
            if move_on_board(move, N) and board[move]=='.':
                to_empty_space=move

                # add board configs for possible moves
                new_board = board.copy()        
                new_board[tuple(pichu_pos)]='.'

                new_row = move[0]
                if new_row == last_row:
                    new_board[move]=raichu
                else:
                    new_board[move]=pichu

                pichu_boards.append(new_board)

        #check two diagonal spaces ahead
        diag_2_l = np.array([forward*2,2])
        diag_2_r = np.array([forward*2,-2])
        diag_2 = np.vstack([np.add(pichu_pos, diag_2_l),
                            np.add(pichu_pos, diag_2_r)])
        diag_1_2 = np.concatenate((diag_1, diag_2), axis = 1).reshape(-1, 2,2)
        diag_1_2_moves = [[tuple(move[0]), tuple(move[1])] for move in diag_1_2]

        #if pichu can jump over opp pichu to an empty space, add to possible moves list

        for move in diag_1_2_moves:
            if move_on_board(move[1], N):
                jump_avail = board[move[0]]+board[move[1]]==opp_pichu+'.'

                if jump_avail:
                    pichu_jump =move[1]
                    new_board = board.copy()        
                    new_board[tuple(pichu_pos)]='.'
                    new_board[move[0]]='.'

                    new_row = pichu_jump[0]
                    if new_row == last_row:
                        new_board[pichu_jump]=raichu
                    else:
                        new_board[pichu_jump]=pichu

                    pichu_boards.append(new_board)
    

    return pichu_boards

In [15]:
board

array([['.', '.', '.', '.', '.', '.', '.', '.'],
       ['W', 'b', 'W', '.', 'W', '.', 'W', '.'],
       ['.', 'w', '.', 'w', '.', 'w', '.', 'w'],
       ['.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', 'w', '.', '.', '.', '.', '.', '.'],
       ['b', '.', 'b', '.', 'b', '.', 'b', '.'],
       ['.', 'B', 'w', 'B', '.', 'B', '.', 'B'],
       ['.', '.', '.', '.', '.', '.', '.', '.']], dtype='<U1')

In [14]:
print(len(possible_pichu_moves(board)))
possible_pichu_moves(board)

9


[array([['.', '.', '$', '.', '.', '.', '.', '.'],
        ['W', '.', 'W', '.', 'W', '.', 'W', '.'],
        ['.', 'w', '.', 'w', '.', 'w', '.', 'w'],
        ['.', '.', '.', '.', '.', '.', '.', '.'],
        ['.', 'w', '.', '.', '.', '.', '.', '.'],
        ['b', '.', 'b', '.', 'b', '.', 'b', '.'],
        ['.', 'B', 'w', 'B', '.', 'B', '.', 'B'],
        ['.', '.', '.', '.', '.', '.', '.', '.']], dtype='<U1'),
 array([['$', '.', '.', '.', '.', '.', '.', '.'],
        ['W', '.', 'W', '.', 'W', '.', 'W', '.'],
        ['.', 'w', '.', 'w', '.', 'w', '.', 'w'],
        ['.', '.', '.', '.', '.', '.', '.', '.'],
        ['.', 'w', '.', '.', '.', '.', '.', '.'],
        ['b', '.', 'b', '.', 'b', '.', 'b', '.'],
        ['.', 'B', 'w', 'B', '.', 'B', '.', 'B'],
        ['.', '.', '.', '.', '.', '.', '.', '.']], dtype='<U1'),
 array([['.', '.', '.', '.', '.', '.', '.', '.'],
        ['W', 'b', 'W', '.', 'W', '.', 'W', '.'],
        ['.', 'w', '.', 'w', '.', 'w', '.', 'w'],
        ['.', '.', '

In [16]:
#select player
player = 'w'

#set player variables
if player == 'w':
    forward, backward, last_row  = 1, -1, N-1
    pichu, pikachu, raichu = 'w', 'W', '@'
    opp_pichu, opp_pikachu, opp_raichu = 'b', 'B', '$'

if player == 'b':
    forward, backward, last_row  = -1, 1, 0
    pichu, pikachu, raichu = 'b', 'B', '$'
    opp_pichu, opp_pikachu, opp_raichu = 'w', 'W', '@'

In [17]:
def possible_pikachu_moves(board):
    pikachu_boards = []
    
    #find player pikachus
    row, col = np.where(board == pikachu)
    all_pikachus = np.column_stack((row, col))

    all_moves = []
    move_range = 3
    forward_moves = [(x*forward,0) for x in range(1,move_range+1)]
    backward_moves=[(x*backward, 0) for x in range(1, move_range+1)]
    right_moves = [(0,x) for x in range(1, move_range+1)]
    left_moves = [(0,x*-1) for x in range(1, move_range+1)]
    all_moves.append(forward_moves)
    all_moves.append(backward_moves)
    all_moves.append(right_moves)
    all_moves.append(left_moves)


   
    # pikachu moves to empty spaces
    
    moves_1_2 = []
    moves_1_2.append(forward_moves[:2])
    moves_1_2.append(backward_moves[:2])
    moves_1_2.append(left_moves[:2])
    moves_1_2.append(right_moves[:2])
    
    pos_1_2 = []
    for n in range(len(all_pikachus)):
        pos = np.add(all_pikachus[n], moves_1_2)        
        pos_1_2.append([[tuple(x) for x in pos[n]] for n in range(len(moves_1_2))])


    for i, pikachu_pos in enumerate(pos_1_2):
        current_pikachu = tuple(all_pikachus[i])
        for direction in pikachu_pos:
            path = []
            for move in direction:
                if move_on_board(move, N):
                    path.append(move)
            board_view =[board[path[n]] for n in range(len(path))]
            for j , step in enumerate(board_view):
                if step =='.':
                    new_board = board.copy()        
                    new_board[current_pikachu]='.'

                    new_row = path[j][0]
                    if new_row == last_row:
                        new_board[path[j]]=raichu
                    else:
                        new_board[path[j]]=pikachu
                    pikachu_boards.append(new_board)
                else:
                    break


    pos_1_2_3 = []
    for n in range(len(all_pikachus)):
        pos = np.add(all_pikachus[n], all_moves)        
        pos_1_2_3.append([[tuple(x) for x in pos[n]] for n in range(len(all_moves))])


    # pikachu jumps to empty space
    for i, pikachu_pos in enumerate(pos_1_2_3):
        current_pikachu = tuple(all_pikachus[i])
        for direction in pikachu_pos:
            path = []
            for move in direction:
                if move_on_board(move, N):
                    path.append(move)
            board_view =[board[path[n]] for n in range(len(path))]

            try:
                if board_view[0] in opp_pichu+opp_pikachu and board_view[1] =='.':
                    new_board = board.copy()        
                    new_board[current_pikachu]='.'
                    new_board[path[0]]='.'

                    new_row = path[1][0]
                    if new_row == last_row:
                        new_board[path[1]]=raichu
                    else:
                        new_board[path[1]]=pikachu
                    pikachu_boards.append(new_board)
            except:
                pass

            try:
                if board_view[0] in opp_pichu+opp_pikachu and board_view[1]=='.' and board_view[2]=='.':
                    new_board = board.copy()        
                    new_board[current_pikachu]='.'
                    new_board[path[0]]='.'

                    new_row = path[2][0]
                    if new_row == last_row:
                        new_board[path[2]]=raichu
                    else:
                        new_board[path[2]]=pikachu
                    pikachu_boards.append(new_board)
            except:
                pass

            try:
                if board_view[0] == '.' and board_view[1] in opp_pichu+opp_pikachu and board_view[2]=='.':
                    new_board = board.copy()        
                    new_board[current_pikachu]='.'
                    new_board[path[1]]='.'

                    new_row = path[2][0]
                    if new_row == last_row:
                        new_board[path[2]]=raichu
                    else:
                        new_board[path[2]]=pikachu
                    pikachu_boards.append(new_board)
            except:
                pass

    return pikachu_boards
        


In [18]:
board

array([['.', '.', '.', '.', '.', '.', '.', '.'],
       ['W', 'b', 'W', '.', 'W', '.', 'W', '.'],
       ['.', 'w', '.', 'w', '.', 'w', '.', 'w'],
       ['.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', 'w', '.', '.', '.', '.', '.', '.'],
       ['b', '.', 'b', '.', 'b', '.', 'b', '.'],
       ['.', 'B', 'w', 'B', '.', 'B', '.', 'B'],
       ['.', '.', '.', '.', '.', '.', '.', '.']], dtype='<U1')

In [20]:
print(len(possible_pikachu_moves(board)))
possible_pikachu_moves(board)

17


[array([['.', '.', '.', '.', '.', '.', '.', '.'],
        ['.', 'b', 'W', '.', 'W', '.', 'W', '.'],
        ['W', 'w', '.', 'w', '.', 'w', '.', 'w'],
        ['.', '.', '.', '.', '.', '.', '.', '.'],
        ['.', 'w', '.', '.', '.', '.', '.', '.'],
        ['b', '.', 'b', '.', 'b', '.', 'b', '.'],
        ['.', 'B', 'w', 'B', '.', 'B', '.', 'B'],
        ['.', '.', '.', '.', '.', '.', '.', '.']], dtype='<U1'),
 array([['.', '.', '.', '.', '.', '.', '.', '.'],
        ['.', 'b', 'W', '.', 'W', '.', 'W', '.'],
        ['.', 'w', '.', 'w', '.', 'w', '.', 'w'],
        ['W', '.', '.', '.', '.', '.', '.', '.'],
        ['.', 'w', '.', '.', '.', '.', '.', '.'],
        ['b', '.', 'b', '.', 'b', '.', 'b', '.'],
        ['.', 'B', 'w', 'B', '.', 'B', '.', 'B'],
        ['.', '.', '.', '.', '.', '.', '.', '.']], dtype='<U1'),
 array([['W', '.', '.', '.', '.', '.', '.', '.'],
        ['.', 'b', 'W', '.', 'W', '.', 'W', '.'],
        ['.', 'w', '.', 'w', '.', 'w', '.', 'w'],
        ['.', '.', '

In [23]:
def possible_raichu_boards(board):
    raichu_boards = []
    
    #find player raichus
    row, col = np.where(board == raichu)
    all_raichus = np.column_stack((row, col))


    all_moves = []
    move_range = N
    forward_moves = [(x*forward,0) for x in range(1,move_range)]
    backward_moves=[(x*backward, 0) for x in range(1, move_range)]
    right_moves = [(0,x) for x in range(1, move_range)]
    left_moves = [(0,x*-1) for x in range(1, move_range)]
    diag_II = [(x*forward, x) for x in range(1, move_range)]
    diag_I = [(x*backward, x) for x in range(1, move_range)]
    diag_III = [(x*backward, x*-1) for x in range(1, move_range)]
    diag_IV = [(x*forward, x*-1) for x in range(1, move_range)]
    all_moves.append(forward_moves)
    all_moves.append(backward_moves)
    all_moves.append(right_moves)
    all_moves.append(left_moves)
    all_moves.append(diag_I)
    all_moves.append(diag_II)
    all_moves.append(diag_III)
    all_moves.append(diag_IV)


    all_pos = []
    for raichu_pos in all_raichus:
        pos = np.add(raichu_pos, all_moves)
        all_pos.append([[tuple(x) for x in pos[n]] for n in range(len(all_moves))])


    # raichu moves to empty spaces
    for i, raichu_pos in enumerate(all_pos):
        current_raichu = tuple(all_raichus[i])
        for direction in raichu_pos:
            path = []
            for move in direction:
                if move_on_board(move, N):
                    path.append(move)
            board_view =[board[path[n]] for n in range(len(path))]
            for j , step in enumerate(board_view):
                if step =='.':
                    new_board = board.copy()        
                    new_board[current_raichu]='.'
                    new_board[path[j]]=raichu
                    raichu_boards.append(new_board)
                else:
                    break


    # pikachu jumps to empty space
    for i, raichu_pos in enumerate(all_pos):
        current_raichu = tuple(all_raichus[i])
        for direction in raichu_pos:
            path = []
            for move in direction:
                if move_on_board(move, N):
                    path.append(move)
            board_view =[board[path[n]] for n in range(len(path))]


            for step in board_view:
                if step in opp_pichu+opp_pikachu+opp_raichu:
                    first_opp = board_view.index(step)
                    before_first_opp = board_view[:first_opp]
                    empty_before = all([x == '.' for x in before_first_opp])
                    if not empty_before:
                        break
                    else:
                        after = board_view[first_opp:]
                        if len(after)==1:
                            break
                        else:
                            for n, step in enumerate(after[1:],1):
                                if step == '.':
                                    new_board = board.copy()        
                                    new_board[current_raichu]='.'
                                    new_board[path[first_opp]]='.'
                                    new_board[path[first_opp+n]]=raichu
                                    raichu_boards.append(new_board)
                                else:
                                    break
                        break
                
    return raichu_boards

In [24]:
#select player
player = 'w'

#set player variables
if player == 'w':
    forward, backward, last_row  = 1, -1, N-1
    pichu, pikachu, raichu = 'w', 'W', '@'
    opp_pichu, opp_pikachu, opp_raichu = 'b', 'B', '$'

if player == 'b':
    forward, backward, last_row  = -1, 1, 0
    pichu, pikachu, raichu = 'b', 'B', '$'
    opp_pichu, opp_pikachu, opp_raichu = 'w', 'W', '@'

In [25]:
board[(1,1)]='@'
board[(4,4)]='@'
board[(2,5)]='$'
board[(3,7)]='$'
board

array([['.', '.', '.', '.', '.', '.', '.', '.'],
       ['W', '@', 'W', '.', 'W', '.', 'W', '.'],
       ['.', 'w', '.', 'w', '.', '$', '.', 'w'],
       ['.', '.', '.', '.', '.', '.', '.', '$'],
       ['.', 'w', '.', '.', '@', '.', '.', '.'],
       ['b', '.', 'b', '.', 'b', '.', 'b', '.'],
       ['.', 'B', 'w', 'B', '.', 'B', '.', 'B'],
       ['.', '.', '.', '.', '.', '.', '.', '.']], dtype='<U1')

In [26]:
board

array([['.', '.', '.', '.', '.', '.', '.', '.'],
       ['W', '@', 'W', '.', 'W', '.', 'W', '.'],
       ['.', 'w', '.', 'w', '.', '$', '.', 'w'],
       ['.', '.', '.', '.', '.', '.', '.', '$'],
       ['.', 'w', '.', '.', '@', '.', '.', '.'],
       ['b', '.', 'b', '.', 'b', '.', 'b', '.'],
       ['.', 'B', 'w', 'B', '.', 'B', '.', 'B'],
       ['.', '.', '.', '.', '.', '.', '.', '.']], dtype='<U1')

In [29]:
print(len(possible_raichu_boards(board)))
possible_raichu_boards(board)

24


[array([['.', '@', '.', '.', '.', '.', '.', '.'],
        ['W', '.', 'W', '.', 'W', '.', 'W', '.'],
        ['.', 'w', '.', 'w', '.', '$', '.', 'w'],
        ['.', '.', '.', '.', '.', '.', '.', '$'],
        ['.', 'w', '.', '.', '@', '.', '.', '.'],
        ['b', '.', 'b', '.', 'b', '.', 'b', '.'],
        ['.', 'B', 'w', 'B', '.', 'B', '.', 'B'],
        ['.', '.', '.', '.', '.', '.', '.', '.']], dtype='<U1'),
 array([['.', '.', '@', '.', '.', '.', '.', '.'],
        ['W', '.', 'W', '.', 'W', '.', 'W', '.'],
        ['.', 'w', '.', 'w', '.', '$', '.', 'w'],
        ['.', '.', '.', '.', '.', '.', '.', '$'],
        ['.', 'w', '.', '.', '@', '.', '.', '.'],
        ['b', '.', 'b', '.', 'b', '.', 'b', '.'],
        ['.', 'B', 'w', 'B', '.', 'B', '.', 'B'],
        ['.', '.', '.', '.', '.', '.', '.', '.']], dtype='<U1'),
 array([['.', '.', '.', '.', '.', '.', '.', '.'],
        ['W', '.', 'W', '.', 'W', '.', 'W', '.'],
        ['.', 'w', '@', 'w', '.', '$', '.', 'w'],
        ['.', '.', '

In [30]:
# check if terminal state
def is_terminal_state(board):

    min_pieces = 'bB$'  
    max_pieces='wW@'

    unique, counts = np.unique(board, return_counts = True)
    pieces_on_board = dict(zip(unique, counts))
    
    max_total = 0
    for piece in max_pieces:
        try:
            max_total+=pieces_on_board[piece]
        except:
            continue

    min_total = 0
    for piece in min_pieces:
        try:
            min_total += pieces_on_board[piece]
        except:
            continue
    return (max_total == 0  or min_total ==0, min_total, max_total)

In [31]:
is_terminal_state(board)

(False, 10, 11)

In [ ]:
# find leaves at horizon

In [21]:
# evaluation function

In [ ]:
# max function

In [ ]:
# min function

In [22]:
# minimax function

In [1]:
###  given (N, player (w or b), board, timelimit (sec)) return the next best move for the player
def find_best_move(board, N, player, timelimit):
    # This sample code just returns the same board over and over again (which
    # isn't a valid move anyway.) Replace this with your code!
    #
    while True:
        time.sleep(1)
        yield board

In [35]:
board = initial_board
N = 8
player = 'w'
timelimit = 1

In [36]:
if __name__ == "__main__":
    if len('xxxxx') != 5:
        raise Exception("Usage: Raichu.py N player board timelimit")
        
    (_, N, player, board, timelimit) = 'x', N, player, board, timelimit
    N=int(N)
    timelimit=int(timelimit)
    if player not in "wb":
        raise Exception("Invalid player.")

    if len(board) != N*N or 0 in [c in "wb.WB@$" for c in board]:
        raise Exception("Bad board string.")

    print("Searching for best move for " + player + " from board state: \n" + board_to_string(board, N))
    print("Here's what I decided:")
    for new_board in find_best_move(board, N, player, timelimit):
        print(new_board)

Searching for best move for w from board state: 
........
W.W.W.W.
.w.w.w.w
........
........
b.b.b.b.
.B.B.B.B
........
Here's what I decided:
........W.W.W.W..w.w.w.w................b.b.b.b..B.B.B.B........
........W.W.W.W..w.w.w.w................b.b.b.b..B.B.B.B........
........W.W.W.W..w.w.w.w................b.b.b.b..B.B.B.B........
........W.W.W.W..w.w.w.w................b.b.b.b..B.B.B.B........
........W.W.W.W..w.w.w.w................b.b.b.b..B.B.B.B........
........W.W.W.W..w.w.w.w................b.b.b.b..B.B.B.B........
........W.W.W.W..w.w.w.w................b.b.b.b..B.B.B.B........
........W.W.W.W..w.w.w.w................b.b.b.b..B.B.B.B........
........W.W.W.W..w.w.w.w................b.b.b.b..B.B.B.B........
........W.W.W.W..w.w.w.w................b.b.b.b..B.B.B.B........
........W.W.W.W..w.w.w.w................b.b.b.b..B.B.B.B........
........W.W.W.W..w.w.w.w................b.b.b.b..B.B.B.B........


KeyboardInterrupt: 